In [ ]:
%whos

In [1]:
key = pd.read_csv('../data/key.csv')
train = pd.read_csv('../data/train.csv')
weather = pd.read_csv('../data/weather.csv')
test = pd.read_csv('../data/test.csv')

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [2]:
train['log_units'] = np.log(train['units'] + 1)
train.tail()

,date,store_nbr,item_nbr,units,log_units
4617595,2014-10-31,45,107,0,0.0
4617596,2014-10-31,45,108,0,0.0
4617597,2014-10-31,45,109,0,0.0
4617598,2014-10-31,45,110,0,0.0
4617599,2014-10-31,45,111,0,0.0


In [3]:
train_new = train[:2255853] # 2013년 3월 31일까지의 train data

In [4]:
weather_train = weather[:8968] # 2013년 3월 31일까지의 weather data

### weather 관련 functions

In [5]:
def get_weather_train(df, station_nbr) : # staion_nbr에 따라서 weather train dataframe을 만드는 함수
    new_df = df[df['station_nbr'] == station_nbr]
    new_df.reset_index(drop = True, inplace = True)
    return new_df    

In [6]:
def set_tmax(df) : # tmax의 missing value를 처리하는 함수
    for num in range(len(df)) :  # tmax의 missing value 처리,,, 이틀 전날의 0.2배 + 하루 전날의 0.8배
        if df.at[num, 'tmax'] == 'M' :
            df.set_value(num, 'tmax', int(0.2 * int(df.at[num - 2, 'tmax']) + 0.8 * int(df.at[num - 1, 'tmax'])))
        
        else :
            df.at[num, 'tmax'] = int(df.at[num, 'tmax'])

In [7]:
def set_tmin(df) : # tmin의 missing value를 처리하는 함수
    for num in range(len(df)) :  # tmax의 missing value 처리,,, 이틀 전날의 0.2배 + 하루 전날의 0.8배
        if df.at[num, 'tmin'] == 'M' :
            df.set_value(num, 'tmin', int(0.2 * int(df.at[num - 2, 'tmin']) + 0.8 * int(df.at[num - 1, 'tmin'])))
            
        else :
            df.at[num,'tmin'] = int(df.at[num, 'tmin'])

In [8]:
def set_tavg(df) : # tavg의 값을 처리하는 함수
    for num in range(len(df)) :
        if df.at[num, 'tavg'] == 'M' :
            df.set_value(num, 'tavg', int((df.at[num, 'tmax'] + df.at[num, 'tmin']) / 2)) # (tmax + tmin) / 2
            
        else :
            df.at[num, 'tavg'] = int(df.at[num, 'tavg'])   

In [9]:
def set_depart(df) : # how can I get depart value..wtf
    if df.at[0, 'depart'] == 'M' :
        print("Cannot get depart value") # depart값을 구할 수 없는 경우
            
    else :
        depart_base = int(df.at[0, 'tavg']) - int(df.at[0, 'depart']) # 각 weather station을 보면 tavg - depart값이 일정하다.. 개이득..
        for num in range(1, len(df)) :
            df.set_value(num, 'depart', int((df.at[num, 'tavg'] - depart_base)))

In [10]:
def set_dewpoint(df) : 
    for num in range(len(df)) :
        if df.at[num, 'dewpoint'] == 'M' :
            df.set_value(num, 'dewpoint', np.nan)

In [11]:
def set_wetbulb(df) : # 습구온도는 불포화공기에서는 건구온도보다 낮고 노점(이슬점, dewpoint)보다 높지만, 포화공기에서 3자는 일치한다.
    for num in range(len(df)) :
        if df.at[num, 'wetbulb'] == 'M' :
            df.set_value(num, 'wetbulb', np.nan)

In [12]:
def set_heat_cool(df) : # heat, cool의 missing value 처리
    for num in range(len(df)) :
        if df.at[num, 'heat'] == 'M' or df.at[num, 'cool'] == 'M' :
            
            if df.at[num, 'tavg'] == 65 : # tavg == 65 이면 heat, cool 모두 0
                df.set_value(num, 'heat', 0)
                df.set_value(num, 'cool', 0)
            
            elif df.at[num, 'tavg'] > 65 : #tavg > 65 이면 heat = 0, cool = tavg - 65
                df.set_value(num, 'heat', 0)
                df.set_value(num, 'cool', df.at[num, 'tavg'] - 65)
            
            else : #tavg < 65 이면 heat = 65 - tavg, cool = 0
                df.set_value(num, 'heat', 65 - df.at[num, 'tavg'])
                df.set_value(num, 'cool', 0)

In [13]:
def merge_heat_cool(df) :
    df['heat_cool'] = ""
    
    for num in range(len(df)) :
        df.set_value(num, "heat_cool", int(df.at[num, "cool"]) - int(df.at[num, "heat"]))

In [14]:
def set_snowfall_preciptotal(df) : #codesum에 RA가 있는데 preciptotal이 T인경우도있다...
    for num in range(len(df)) :
        if ('T' in df.at[num, 'snowfall']) or ('M' in df.at[num, 'snowfall']) : # snowfall이 trace or M이면 0으로 setting
            df.set_value(num, 'snowfall', 0.0)
        
        if ('T' in df.at[num, 'preciptotal']) or ('M' in df.at[num, 'preciptotal']) : # preciptotal이 trace or M이면 0으로 setting
            df.set_value(num, 'preciptotal', 0.0)

In [15]:
def set_stnpressure(df) :
    for num in range(len(df)) :
        if df.at[num, 'stnpressure'] == 'M' :
            df.set_value(num, 'stnpressure', np.nan)

In [16]:
def set_sealevel(df) :
    for num in range(len(df)) :
        if df.at[num, 'sealevel'] == 'M' :
            df.set_value(num, 'sealevel', np.nan)

In [17]:
def set_resultspeed(df) : # 바람관련 column 3개는 어떤 연관관계가 있지않을까..
#     The fastest 2-minute (average) wind speed, in Miles Per Hour.
    for num in range(len(df)) :
        if df.at[num, 'resultspeed'] == 'M' :
            df.set_value(num, 'resultspeed', np.nan)

In [18]:
def set_resultdir(df) :
#     resultdir : The direction the fastest 2-minute wind was blowing FROM in tens of degrees
#     27 would be 270 degrees = wind was blowing from the west to the east
#     (36 = North, 09 = East, 18 = South, 27 = West)
    for num in range(len(df)) :
        if df.at[num, 'resultdir'] == 'M' :
            df.set_value(num, 'resultdir', np.nan)

In [19]:
def set_avgspeed(df) :
#     The average wind speed for the day, in Miles Per Hour
    for num in range(len(df)) :
        if df.at[num, 'avgspeed'] == 'M' :
            df.set_value(num, 'avgspeed', np.nan)

### train 관련 functions

In [20]:
def get_train_train(df, store_nbr) : # store_nbr에 따라 train data를 나눠주는 함수
    new_df = df[df['store_nbr'] == store_nbr]
    new_df.reset_index(drop = True, inplace = True)
    return new_df

### weather와 train data를 합쳐주는 함수
- train data 중 log_units != 0 인 item들만 합쳐줌.. (log_units != 0 이면 units != 0)

In [21]:
def merge_weather_train(weather, train) :
    train_pivot = train.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
    train_pivot = train_pivot.loc[:, (train_pivot != 0).any(axis = 0)] # log_units이 모두 0인 item_nbr(column)을 삭제..
    train_pivot.loc['2012-12-25'] = 0 # 2012-12-25의 data를 0으로 추가... 원래 없었던 data이므로 0으로 설정함
    
    list_item_nbr = train_pivot.columns # units 전체가 0이 아닌 item_nbr들을 list형태로 받음
#     length = len(train_pivot.columns) # units전체가 0이 아닌 item_nbr이 총 몇개 있는지..
        
    train_pivot.index.name = "date"
    train_pivot.reset_index(inplace = True)
    train_pivot.sort_values(by = 'date', inplace = True)
    train_pivot.reset_index(drop = True, inplace = True)
    
    for num in list_item_nbr :
        weather[num] = train_pivot[num]
    
    return weather

In [22]:
def get_item_nbr(df) : # 모든 units이 0이 아닌 item_nbr을 구하는 함수, list형태로 return
    tmp = df.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
    tmp = tmp.loc[:, (tmp != 0).any(axis = 0)]
    tmp.loc['2012-12-25'] = 0 # 2012-12-25가 빠져있음 train data에서.. 그래서 log_units = 0으로 넣어줌.
    
    tmp.reset_index(inplace = True)
    tmp.sort_values(by = 'date', inplace = True)
    tmp.drop(['date'], axis = 1, inplace = True)
    
    result = list(tmp.columns)
    
    return result

In [ ]:
#station_nbr로 weather_train을 나눠줌
# 박두진 강사님이 이 부분을 줄일 수 있는 방법을 찾아봐주신다 하였음..일단은 그냥 쓰자!

weather_train_1 = get_weather_train(weather_train, 1)
weather_train_2 = get_weather_train(weather_train, 2)
weather_train_3 = get_weather_train(weather_train, 3)
weather_train_4 = get_weather_train(weather_train, 4)
weather_train_5 = get_weather_train(weather_train, 5) # missing value가 많아서 일단은..
weather_train_6 = get_weather_train(weather_train, 6)
weather_train_7 = get_weather_train(weather_train, 7)
weather_train_8 = get_weather_train(weather_train, 8)
weather_train_9 = get_weather_train(weather_train, 9)
weather_train_10 = get_weather_train(weather_train, 10)
weather_train_11 = get_weather_train(weather_train, 11)
weather_train_12 = get_weather_train(weather_train, 12)
weather_train_13 = get_weather_train(weather_train, 13)
weather_train_14 = get_weather_train(weather_train, 14)
weather_train_15 = get_weather_train(weather_train, 15)
weather_train_16 = get_weather_train(weather_train, 16)
weather_train_17 = get_weather_train(weather_train, 17)
weather_train_18 = get_weather_train(weather_train, 18)
weather_train_19 = get_weather_train(weather_train, 19)
weather_train_20 = get_weather_train(weather_train, 20)

In [ ]:
train_new_1 = get_train_train(train_new, 1)
train_new_2 = get_train_train(train_new, 2)
train_new_3 = get_train_train(train_new, 3)
train_new_4 = get_train_train(train_new, 4)
train_new_5 = get_train_train(train_new, 5)
train_new_6 = get_train_train(train_new, 6)
train_new_7 = get_train_train(train_new, 7)
train_new_8 = get_train_train(train_new, 8)
train_new_9 = get_train_train(train_new, 9)
train_new_10 = get_train_train(train_new, 10)
train_new_11 = get_train_train(train_new, 11)
train_new_12 = get_train_train(train_new, 12)
train_new_13 = get_train_train(train_new, 13)
train_new_14 = get_train_train(train_new, 14)
train_new_15 = get_train_train(train_new, 15)
train_new_16 = get_train_train(train_new, 16)
train_new_17 = get_train_train(train_new, 17)
train_new_18 = get_train_train(train_new, 18)
train_new_19 = get_train_train(train_new, 19)
train_new_20 = get_train_train(train_new, 20)
train_new_21 = get_train_train(train_new, 21)
train_new_22 = get_train_train(train_new, 22)
train_new_23 = get_train_train(train_new, 23)
train_new_24 = get_train_train(train_new, 24)
train_new_25 = get_train_train(train_new, 25)
train_new_26 = get_train_train(train_new, 26)
train_new_27 = get_train_train(train_new, 27)
train_new_28 = get_train_train(train_new, 28)
train_new_29 = get_train_train(train_new, 29)
train_new_30 = get_train_train(train_new, 30)
train_new_31 = get_train_train(train_new, 31)
train_new_32 = get_train_train(train_new, 32)
train_new_33 = get_train_train(train_new, 33)
train_new_34 = get_train_train(train_new, 34)
train_new_35 = get_train_train(train_new, 35)
train_new_36 = get_train_train(train_new, 36)
train_new_37 = get_train_train(train_new, 37)
train_new_38 = get_train_train(train_new, 38)
train_new_39 = get_train_train(train_new, 39)
train_new_40 = get_train_train(train_new, 40)
train_new_41 = get_train_train(train_new, 41)
train_new_42 = get_train_train(train_new, 42)
train_new_43 = get_train_train(train_new, 43)
train_new_44 = get_train_train(train_new, 44)
train_new_45 = get_train_train(train_new, 45)

In [23]:
weather_train.sort_values(by = ['station_nbr', 'date'], inplace = True)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
weather_train.reset_index(drop = True, inplace = True)
weather_train.head()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2012-01-01,52,31,42,M,36,40,23,0,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
1,1,2012-01-02,50,31,41,M,26,35,24,0,-,-,,M,0.01,29.44,29.62,9.8,24,10.3
2,1,2012-01-03,32,11,22,M,4,18,43,0,-,-,,M,0.00,29.67,29.87,10.8,31,11.6
3,1,2012-01-04,28,9,19,M,-1,14,46,0,-,-,,M,0.00,29.86,30.03,6.3,27,8.3
4,1,2012-01-05,38,25,32,M,13,25,33,0,-,-,,M,0.00,29.67,29.84,6.9,25,7.8


In [26]:
set_tmax(weather_train)
set_tmin(weather_train)
set_tavg(weather_train)
set_heat_cool(weather_train)
merge_heat_cool(weather_train)
# set_depart(weather_train)
set_snowfall_preciptotal(weather_train)
set_dewpoint(weather_train)
set_wetbulb(weather_train)
set_stnpressure(weather_train)
set_sealevel(weather_train)
set_resultspeed(weather_train)
set_resultdir(weather_train)
set_avgspeed(weather_train)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
for num in range(len(weather_train)) :
    if 'M' in weather_train.at[num, 'depart'] :
        weather_train.set_value(num, 'depart', np.NaN)

In [28]:
weather_train.columns

Index(['station_nbr', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'heat_cool'],
      dtype='object')

In [29]:
weather_train_drop = weather_train.drop(['heat', 'cool', 'sunrise', 'sunset', 'codesum'], axis = 1)

In [30]:
train_new.head()

,date,store_nbr,item_nbr,units,log_units
0,2012-01-01,1,1,0,0.0
1,2012-01-01,1,2,0,0.0
2,2012-01-01,1,3,0,0.0
3,2012-01-01,1,4,0,0.0
4,2012-01-01,1,5,0,0.0


In [31]:
df = pd.merge(weather_train_drop, key, on = 'station_nbr')

In [32]:
df_ = pd.merge(df, train_new, on = ['date', 'store_nbr'])

In [33]:
df_.head()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,heat_cool,store_nbr,item_nbr,units,log_units
0,1,2012-01-01,52,31,42,NaN,36,40,0,0.05,29.78,29.92,3.6,20,4.6,-23,1,1,0,0.0
1,1,2012-01-01,52,31,42,NaN,36,40,0,0.05,29.78,29.92,3.6,20,4.6,-23,1,2,0,0.0
2,1,2012-01-01,52,31,42,NaN,36,40,0,0.05,29.78,29.92,3.6,20,4.6,-23,1,3,0,0.0
3,1,2012-01-01,52,31,42,NaN,36,40,0,0.05,29.78,29.92,3.6,20,4.6,-23,1,4,0,0.0
4,1,2012-01-01,52,31,42,NaN,36,40,0,0.05,29.78,29.92,3.6,20,4.6,-23,1,5,0,0.0


In [47]:
weather_train_drop.columns

Index(['station_nbr', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', 'heat_cool'],
      dtype='object')

In [48]:
weather_col = list(weather_train_drop.columns)
weather_col

['station_nbr',
 'date',
 'tmax',
 'tmin',
 'tavg',
 'depart',
 'dewpoint',
 'wetbulb',
 'snowfall',
 'preciptotal',
 'stnpressure',
 'sealevel',
 'resultspeed',
 'resultdir',
 'avgspeed',
 'heat_cool']

In [49]:
category = ['station_nbr', 'item_nbr']
weather_col = weather_col[2:]

In [50]:
formula = ['C(station_nbr):C(item_nbr):{}'.format(weather)  for weather in weather_col]

In [53]:
total_formula = " + ".join(formula)

In [54]:
total_formula

'C(station_nbr):C(item_nbr):tmax + C(station_nbr):C(item_nbr):tmin + C(station_nbr):C(item_nbr):tavg + C(station_nbr):C(item_nbr):depart + C(station_nbr):C(item_nbr):dewpoint + C(station_nbr):C(item_nbr):wetbulb + C(station_nbr):C(item_nbr):snowfall + C(station_nbr):C(item_nbr):preciptotal + C(station_nbr):C(item_nbr):stnpressure + C(station_nbr):C(item_nbr):sealevel + C(station_nbr):C(item_nbr):resultspeed + C(station_nbr):C(item_nbr):resultdir + C(station_nbr):C(item_nbr):avgspeed + C(station_nbr):C(item_nbr):heat_cool'

In [55]:
total = 'log_units ~ ' + '0 + '+ total_formula

In [56]:
total

'log_units ~ 0 + C(station_nbr):C(item_nbr):tmax + C(station_nbr):C(item_nbr):tmin + C(station_nbr):C(item_nbr):tavg + C(station_nbr):C(item_nbr):depart + C(station_nbr):C(item_nbr):dewpoint + C(station_nbr):C(item_nbr):wetbulb + C(station_nbr):C(item_nbr):snowfall + C(station_nbr):C(item_nbr):preciptotal + C(station_nbr):C(item_nbr):stnpressure + C(station_nbr):C(item_nbr):sealevel + C(station_nbr):C(item_nbr):resultspeed + C(station_nbr):C(item_nbr):resultdir + C(station_nbr):C(item_nbr):avgspeed + C(station_nbr):C(item_nbr):heat_cool'

In [57]:
df_.sort_values(by = ['station_nbr', 'date', 'store_nbr', 'item_nbr'], inplace = True)

In [58]:
df_.reset_index(drop = True, inplace = True)

In [ ]:
df_.columns

In [ ]:
df_.head()

In [60]:
model = sm.OLS.from_formula(total, data = df_, missing = 'drop')

KeyboardInterrupt: 

In [ ]:
result = model.fit()
print(result.summary())

## 각 dataframe별로 뒤에 붙는 숫자에 주의해야함
- station_nbr 기준인지 store_nbr 기준인지...

### station_nbr별 특징
- no depart value : 1, 7, 8, 9, 10, 12, 13, 16, 17, 20
    - drop??

### station_nbr == 1
- store_nbr == 1
- no depart value

In [ ]:
set_tmax(weather_train_1)
set_tmin(weather_train_1)
set_tavg(weather_train_1)
set_heat_cool(weather_train_1)
merge_heat_cool(weather_train_1)
set_depart(weather_train_1)
set_snowfall_preciptotal(weather_train_1)
set_dewpoint(weather_train_1)
set_wetbulb(weather_train_1)
set_stnpressure(weather_train_1)
set_sealevel(weather_train_1)
set_resultspeed(weather_train_1)
set_resultdir(weather_train_1)
set_avgspeed(weather_train_1)

# merged_1 = merge_weather_train(weather_train_1, train_new_1)
item_nbr_list_1 = get_item_nbr(train_new_1)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_1 :
    plt.subplot(len(item_nbr_list_1), 1, plotcount)
    plt.scatter(x = merged_1['tmax'], y = merged_1[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount += 1

In [ ]:
sns.pairplot(merged_1, x_vars = ['tmax', 'tmin', 'tavg'], y_vars = item_nbr_list_1)

In [ ]:
weather_train_1_test = weather_train_1.drop(['station_nbr', 'date', 'depart', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

train_new_1_pivot = train_new_1.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# train_new_1_pivot.loc[len(train_new_1)] = 0
# train_new_1_pivot.reset_index(drop = True, inplace = True)

# test = train_new_1.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
train_new_1_pivot = pd.DataFrame(np.insert(train_new_1_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_1 :
    dfX = sm.add_constant(weather_train_1_test)
    dfy = pd.DataFrame(np.asarray(train_new_1_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

In [ ]:
# linear regression에서는 array형태 data type이 필요하다.. np.asarray
# NaN값이 있으면 안되나벼..

weather_train_1_test_array = np.asarray(weather_train_1_test)

for num in item_nbr_list_1 :

    model2 = LinearRegression().fit(weather_train_1_test_array, np.asarray(train_new_1_pivot[num]))

    # model2.coef_
    # model2.intercept_

    predictions = model2.predict(np.asarray(weather_train_1_test_array))

    plt.scatter(train_new_1_pivot[num], predictions)
    # plt.xlabel(u"실제 집값")
    # plt.ylabel(u"집값 예측치")
    plt.show()

###  station_nbr == 2
- store_nbr == 16

In [ ]:
set_tmax(weather_train_2)
set_tmin(weather_train_2)
set_tavg(weather_train_2)
set_heat_cool(weather_train_2)
merge_heat_cool(weather_train_2)
set_depart(weather_train_2)
set_snowfall_preciptotal(weather_train_2)
set_dewpoint(weather_train_2)
set_wetbulb(weather_train_2)
set_stnpressure(weather_train_2)
set_sealevel(weather_train_2)
set_resultspeed(weather_train_2)
set_resultdir(weather_train_2)
set_avgspeed(weather_train_2)

# merged_2 = merge_weather_train(weather_train_2, train_new_16)
item_nbr_list_2 = get_item_nbr(train_new_16)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_2 :
    plt.subplot(len(item_nbr_list_2), 1, plotcount)
    plt.scatter(x = merged_2['tmax'], y = merged_2[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount += 1

In [ ]:
weather_train_2_test = weather_train_2.drop(['station_nbr', 'date', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

train_new_16_pivot = train_new_16.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# train_new_16_pivot.loc[len(train_new_16)] = 0
# train_new_16_pivot.reset_index(drop = True, inplace = True)

train_new_16_pivot = pd.DataFrame(np.insert(train_new_16_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_2 :
    dfX = sm.add_constant(weather_train_1_test)
    dfy = pd.DataFrame(np.asarray(train_new_16_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 3
- store_nbr == 21, 29, 33

In [ ]:
result = pd.concat([train_new_21, train_new_29, train_new_33])

set_tmax(weather_train_3)
set_tmin(weather_train_3)
set_tavg(weather_train_3)
set_heat_cool(weather_train_3)
merge_heat_cool(weather_train_3)
set_depart(weather_train_3)
set_snowfall_preciptotal(weather_train_3)
set_dewpoint(weather_train_3)
set_wetbulb(weather_train_3)
set_stnpressure(weather_train_3)
set_sealevel(weather_train_3)
set_resultspeed(weather_train_3)
set_resultdir(weather_train_3)
set_avgspeed(weather_train_3)

# merged_3 = merge_weather_train(weather_train_3, result)
item_nbr_list_3 = get_item_nbr(result)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_3 :
    plt.subplot(len(item_nbr_list_3), 1, plotcount)
    plt.scatter(x = merged_3['tmax'], y = merged_3[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_3_test = weather_train_3.drop(['station_nbr', 'date', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

result_pivot = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# result_pivot.loc[len(train_new_1)] = 0
# result_pivot.reset_index(drop = True, inplace = True)

result_pivot = pd.DataFrame(np.insert(result_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_3 :
    dfX = sm.add_constant(weather_train_3_test)
    dfy = pd.DataFrame(np.asarray(result_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 4
- store_nbr == 8

In [ ]:
set_tmax(weather_train_4)
set_tmin(weather_train_4)
set_tavg(weather_train_4)
set_heat_cool(weather_train_4)
merge_heat_cool(weather_train_4)
set_depart(weather_train_4)
set_snowfall_preciptotal(weather_train_4)
set_dewpoint(weather_train_4)
set_wetbulb(weather_train_4)
set_stnpressure(weather_train_4)
set_sealevel(weather_train_4)
set_resultspeed(weather_train_4)
set_resultdir(weather_train_4)
set_avgspeed(weather_train_4)

# merged_4 = merge_weather_train(weather_train_4, train_new_8)
item_nbr_list_4 = get_item_nbr(train_new_8)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_4 :
    plt.subplot(len(item_nbr_list_4), 1, plotcount)
    plt.scatter(x = merged_4['tmax'], y = merged_4[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_4_test = weather_train_4.drop(['station_nbr', 'date', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

train_new_8_pivot = train_new_8.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# train_new_8_pivot.loc[len(train_new_8)] = 0
# train_new_8_pivot.reset_index(drop = True, inplace = True)

train_new_8_pivot = pd.DataFrame(np.insert(train_new_8_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_4 :
    dfX = sm.add_constant(weather_train_4_test)
    dfy = pd.DataFrame(np.asarray(train_new_8_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 5
- store_nbr == 35

In [ ]:
# 여긴 좀 고민할 필요가 있음..

### station_nbr == 6
- store_nbr == 7, 13

In [ ]:
result = pd.concat([train_new_7, train_new_13])

set_tmax(weather_train_6)
set_tmin(weather_train_6)
set_tavg(weather_train_6)
set_heat_cool(weather_train_6)
merge_heat_cool(weather_train_6)
set_depart(weather_train_6)
set_snowfall_preciptotal(weather_train_6)
set_dewpoint(weather_train_6)
set_wetbulb(weather_train_6)
set_stnpressure(weather_train_6)
set_sealevel(weather_train_6)
set_resultspeed(weather_train_6)
set_resultdir(weather_train_6)
set_avgspeed(weather_train_6)

# merged_6 = merge_weather_train(weather_train_6, result)
item_nbr_list_6 = get_item_nbr(result)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_6 :
    plt.subplot(len(item_nbr_list_6), 1, plotcount)
    plt.scatter(x = merged_6['tmax'], y = merged_6[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_6_test = weather_train_6.drop(['station_nbr', 'date', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

result_pivot = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# result_pivot.loc[len(result)] = 0
# result_pivot.reset_index(drop = True, inplace = True)

result_pivot = pd.DataFrame(np.insert(result_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_6 :
    dfX = sm.add_constant(weather_train_6_test)
    dfy = pd.DataFrame(np.asarray(result_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 7
- store_nbr == 3, 20, 28
- no depart value

In [ ]:
result = pd.concat([train_new_3, train_new_20, train_new_28])

set_tmax(weather_train_7)
set_tmin(weather_train_7)
set_tavg(weather_train_7)
set_heat_cool(weather_train_7)
merge_heat_cool(weather_train_7)
set_depart(weather_train_7)
set_snowfall_preciptotal(weather_train_7)
set_dewpoint(weather_train_7)
set_wetbulb(weather_train_7)
set_stnpressure(weather_train_7)
set_sealevel(weather_train_7)
set_resultspeed(weather_train_7)
set_resultdir(weather_train_7)
set_avgspeed(weather_train_7)

# merged_7 = merge_weather_train(weather_train_7, result)
item_nbr_list_7 = get_item_nbr(result)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_7 :
    plt.subplot(len(item_nbr_list_7), 1, plotcount)
    plt.scatter(x = merged_7['tmax'], y = merged_7[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_7_test = weather_train_7.drop(['station_nbr', 'date', 'depart', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

result_pivot = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# result_pivot.loc[len(result)] = 0
# result_pivot.reset_index(drop = True, inplace = True)

# test = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
result_pivot = pd.DataFrame(np.insert(result_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_7 :
    dfX = sm.add_constant(weather_train_7_test)
    dfy = pd.DataFrame(np.asarray(result_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 8
- store_nbr == 39
- no depart value

In [ ]:
set_tmax(weather_train_8)
set_tmin(weather_train_8)
set_tavg(weather_train_8)
set_heat_cool(weather_train_8)
merge_heat_cool(weather_train_8)
set_depart(weather_train_8)
set_snowfall_preciptotal(weather_train_8)
set_dewpoint(weather_train_8)
set_wetbulb(weather_train_8)
set_stnpressure(weather_train_8)
set_sealevel(weather_train_8)
set_resultspeed(weather_train_8)
set_resultdir(weather_train_8)
set_avgspeed(weather_train_8)

# merged_8 = merge_weather_train(weather_train_8, train_new_39)
item_nbr_list_8 = get_item_nbr(train_new_39)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_8 :
    plt.subplot(len(item_nbr_list_8), 1, plotcount)
    plt.scatter(x = merged_8['tmax'], y = merged_8[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_8_test = weather_train_8.drop(['station_nbr', 'date', 'depart', 'sunrise', 'sunset', 'codesum',
                                             'heat', 'cool', 'sealevel'], axis = 1)

train_new_39_pivot = train_new_39.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# train_new_39_pivot.loc[len(train_new_39)] = 0
# train_new_39_pivot.reset_index(drop = True, inplace = True)

# test = train_new_39.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
train_new_39_pivot = pd.DataFrame(np.insert(train_new_39_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_8 :
    dfX = sm.add_constant(weather_train_8_test)
    dfy = pd.DataFrame(np.asarray(train_new_39_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 9
- store_nbr == 4, 24
- no depart value

In [ ]:
result = pd.concat([train_new_4, train_new_24])

set_tmax(weather_train_9)
set_tmin(weather_train_9)
set_tavg(weather_train_9)
set_heat_cool(weather_train_9)
merge_heat_cool(weather_train_9)
set_depart(weather_train_9)
set_snowfall_preciptotal(weather_train_9)
set_dewpoint(weather_train_9)
set_wetbulb(weather_train_9)
set_stnpressure(weather_train_9)
set_sealevel(weather_train_9)
set_resultspeed(weather_train_9)
set_resultdir(weather_train_9)
set_avgspeed(weather_train_9)

# merged_9 = merge_weather_train(weather_train_9, result)
item_nbr_list_9 = get_item_nbr(result)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_9 :
    plt.subplot(len(item_nbr_list_9), 1, plotcount)
    plt.scatter(x = merged_9['tmax'], y = merged_9[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_9_test = weather_train_9.drop(['station_nbr', 'date', 'depart', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

result_pivot = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# result_pivot.loc[len(result)] = 0
# result_pivot.reset_index(drop = True, inplace = True)

# test = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
result_pivot = pd.DataFrame(np.insert(result_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_9 :
    dfX = sm.add_constant(weather_train_9_test)
    dfy = pd.DataFrame(np.asarray(result_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 10
- store_nbr == 11, 22, 27
- no depart value

In [ ]:
result = pd.concat([train_new_11, train_new_22, train_new_27])

set_tmax(weather_train_10)
set_tmin(weather_train_10)
set_tavg(weather_train_10)
set_heat_cool(weather_train_10)
merge_heat_cool(weather_train_10)
set_depart(weather_train_10)
set_snowfall_preciptotal(weather_train_10)
set_dewpoint(weather_train_10)
set_wetbulb(weather_train_10)
set_stnpressure(weather_train_10)
set_sealevel(weather_train_10)
set_resultspeed(weather_train_10)
set_resultdir(weather_train_10)
set_avgspeed(weather_train_10)

# merged_10 = merge_weather_train(weather_train_10, result)
item_nbr_list_10 = get_item_nbr(result)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_10 :
    plt.subplot(len(item_nbr_list_10), 1, plotcount)
    plt.scatter(x = merged_10['tmax'], y = merged_10[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_10_test = weather_train_10.drop(['station_nbr', 'date', 'depart', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

result_pivot = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# result_pivot.loc[len(result)] = 0
# result_pivot.reset_index(drop = True, inplace = True)

# test = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
result_pivot = pd.DataFrame(np.insert(result_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_10 :
    dfX = sm.add_constant(weather_train_10_test)
    dfy = pd.DataFrame(np.asarray(result_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 11
- store_nbr == 12, 43

In [ ]:
result = pd.concat([train_new_12, train_new_43])

set_tmax(weather_train_11)
set_tmin(weather_train_11)
set_tavg(weather_train_11)
set_heat_cool(weather_train_11)
merge_heat_cool(weather_train_11)
set_depart(weather_train_11)
set_snowfall_preciptotal(weather_train_11)
set_dewpoint(weather_train_11)
set_wetbulb(weather_train_11)
set_stnpressure(weather_train_11)
set_sealevel(weather_train_11)
set_resultspeed(weather_train_11)
set_resultdir(weather_train_11)
set_avgspeed(weather_train_11)

# merged_11 = merge_weather_train(weather_train_11, result)
item_nbr_list_11 = get_item_nbr(result)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_11 :
    plt.subplot(len(item_nbr_list_11), 1, plotcount)
    plt.scatter(x = merged_11['tmax'], y = merged_11[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_11_test = weather_train_11.drop(['station_nbr', 'date', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

result_pivot = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# result_pivot.loc[len(result)] = 0
# result_pivot.reset_index(drop = True, inplace = True)

# test = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
result_pivot = pd.DataFrame(np.insert(result_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_11 :
    dfX = sm.add_constant(weather_train_11_test)
    dfy = pd.DataFrame(np.asarray(result_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 12
- store_nbr == 5, 10, 41, 44
- no depart value

In [ ]:
result = pd.concat([train_new_5, train_new_10, train_new_41, train_new_44])

set_tmax(weather_train_12)
set_tmin(weather_train_12)
set_tavg(weather_train_12)
set_heat_cool(weather_train_12)
merge_heat_cool(weather_train_12)
set_depart(weather_train_12)
set_snowfall_preciptotal(weather_train_12)
set_dewpoint(weather_train_12)
set_wetbulb(weather_train_12)
set_stnpressure(weather_train_12)
set_sealevel(weather_train_12)
set_resultspeed(weather_train_12)
set_resultdir(weather_train_12)
set_avgspeed(weather_train_12)

# merged_12 = merge_weather_train(weather_train_12, result)
item_nbr_list_12 = get_item_nbr(result)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_12 :
    plt.subplot(len(item_nbr_list_12), 1, plotcount)
    plt.scatter(x = merged_12['tmax'], y = merged_12[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_12_test = weather_train_12.drop(['station_nbr', 'date', 'depart', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

result_pivot = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# result_pivot.loc[len(result)] = 0
# result_pivot.reset_index(drop = True, inplace = True)

# test = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
result_pivot = pd.DataFrame(np.insert(result_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_12 :
    dfX = sm.add_constant(weather_train_12_test)
    dfy = pd.DataFrame(np.asarray(result_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 13
- store_nbr == 15, 25, 32, 37, 40
- no depart value

In [ ]:
result = pd.concat([train_new_15, train_new_25, train_new_32, train_new_37, train_new_40])

set_tmax(weather_train_13)
set_tmin(weather_train_13)
set_tavg(weather_train_13)
set_heat_cool(weather_train_13)
merge_heat_cool(weather_train_13)
set_depart(weather_train_13)
set_snowfall_preciptotal(weather_train_13)
set_dewpoint(weather_train_13)
set_wetbulb(weather_train_13)
set_stnpressure(weather_train_13)
set_sealevel(weather_train_13)
set_resultspeed(weather_train_13)
set_resultdir(weather_train_13)
set_avgspeed(weather_train_13)

# merged_13 = merge_weather_train(weather_train_13, result)
item_nbr_list_13 = get_item_nbr(result)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_13 :
    plt.subplot(len(item_nbr_list_13), 1, plotcount)
    plt.scatter(x = merged_13['tmax'], y = merged_13[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_13_test = weather_train_13.drop(['station_nbr', 'date', 'depart', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

result_pivot = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# result_pivot.loc[len(result)] = 0
# result_pivot.reset_index(drop = True, inplace = True)

# test = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
result_pivot = pd.DataFrame(np.insert(result_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_13 :
    dfX = sm.add_constant(weather_train_13_test)
    dfy = pd.DataFrame(np.asarray(result_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 14
- store_nbr == 2, 6, 38, 42

In [ ]:
result = pd.concat([train_new_2, train_new_6, train_new_38, train_new_42])

set_tmax(weather_train_14)
set_tmin(weather_train_14)
set_tavg(weather_train_14)
set_heat_cool(weather_train_14)
merge_heat_cool(weather_train_14)
set_depart(weather_train_14)
set_snowfall_preciptotal(weather_train_14)
set_dewpoint(weather_train_14)
set_wetbulb(weather_train_14)
set_stnpressure(weather_train_14)
set_sealevel(weather_train_14)
set_resultspeed(weather_train_14)
set_resultdir(weather_train_14)
set_avgspeed(weather_train_14)

# merged_14 = merge_weather_train(weather_train_14, result)
item_nbr_list_14 = get_item_nbr(result)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_14 :
    plt.subplot(len(item_nbr_list_14), 1, plotcount)
    plt.scatter(x = merged_14['tmax'], y = merged_14[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_14_test = weather_train_14.drop(['station_nbr', 'date', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

result_pivot = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# result_pivot.loc[len(result)] = 0
# result_pivot.reset_index(drop = True, inplace = True)

# test = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
result_pivot = pd.DataFrame(np.insert(result_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_14 :
    dfX = sm.add_constant(weather_train_14_test)
    dfy = pd.DataFrame(np.asarray(result_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 15
- store_nbr == 19

In [ ]:
set_tmax(weather_train_15)
set_tmin(weather_train_15)
set_tavg(weather_train_15)
set_heat_cool(weather_train_15)
merge_heat_cool(weather_train_15)
set_depart(weather_train_15)
set_snowfall_preciptotal(weather_train_15)
set_dewpoint(weather_train_15)
set_wetbulb(weather_train_15)
set_stnpressure(weather_train_15)
set_sealevel(weather_train_15)
set_resultspeed(weather_train_15)
set_resultdir(weather_train_15)
set_avgspeed(weather_train_15)

# merged_15 = merge_weather_train(weather_train_15, train_new_19)
item_nbr_list_15 = get_item_nbr(train_new_19)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_15 :
    plt.subplot(len(item_nbr_list_15), 1, plotcount)
    plt.scatter(x = merged_15['tmax'], y = merged_15[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_15_test = weather_train_15.drop(['station_nbr', 'date', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

train_new_19_pivot = train_new_19.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# train_new_19_pivot.loc[len(train_new_19)] = 0
# train_new_19_pivot.reset_index(drop = True, inplace = True)

# test = train_new_19.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
train_new_19_pivot = pd.DataFrame(np.insert(train_new_19_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_15 :
    dfX = sm.add_constant(weather_train_15_test)
    dfy = pd.DataFrame(np.asarray(train_new_19_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 16
- store_nbr == 14, 45
- no depart value

In [ ]:
result = pd.concat([train_new_14, train_new_45])

set_tmax(weather_train_16)
set_tmin(weather_train_16)
set_tavg(weather_train_16)
set_heat_cool(weather_train_16)
merge_heat_cool(weather_train_16)
set_depart(weather_train_16)
set_snowfall_preciptotal(weather_train_16)
set_dewpoint(weather_train_16)
set_wetbulb(weather_train_16)
set_stnpressure(weather_train_16)
set_sealevel(weather_train_16)
set_resultspeed(weather_train_16)
set_resultdir(weather_train_16)
set_avgspeed(weather_train_16)

# merged_16 = merge_weather_train(weather_train_16, result)
item_nbr_list_16 = get_item_nbr(result)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_16 :
    plt.subplot(len(item_nbr_list_16), 1, plotcount)
    plt.scatter(x = merged_16['tmax'], y = merged_16[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_16_test = weather_train_16.drop(['station_nbr', 'date', 'depart', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

result_pivot = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# result_pivot.loc[len(result)] = 0
# result_pivot.reset_index(drop = True, inplace = True)

# test = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
result_pivot = pd.DataFrame(np.insert(result_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_16 :
    dfX = sm.add_constant(weather_train_16_test)
    dfy = pd.DataFrame(np.asarray(result_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 17
- store_nbr == 9, 18, 23, 26, 31, 34
- no depart value

In [ ]:
result = pd.concat([train_new_9, train_new_18, train_new_23, train_new_26, train_new_31, train_new_34])

set_tmax(weather_train_17)
set_tmin(weather_train_17)
set_tavg(weather_train_17)
set_heat_cool(weather_train_17)
merge_heat_cool(weather_train_17)
set_depart(weather_train_17)
set_snowfall_preciptotal(weather_train_17)
set_dewpoint(weather_train_17)
set_wetbulb(weather_train_17)
set_stnpressure(weather_train_17)
set_sealevel(weather_train_17)
set_resultspeed(weather_train_17)
set_resultdir(weather_train_17)
set_avgspeed(weather_train_17)

# merged_17 = merge_weather_train(weather_train_17, result)
item_nbr_list_17 = get_item_nbr(result)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_17 :
    plt.subplot(len(item_nbr_list_17), 1, plotcount)
    plt.scatter(x = merged_17['tmax'], y = merged_17[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_17_test = weather_train_17.drop(['station_nbr', 'date', 'depart', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

result_pivot = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# result_pivot.loc[len(result)] = 0
# result_pivot.reset_index(drop = True, inplace = True)

# test = result.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
result_pivot = pd.DataFrame(np.insert(result_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_17 :
    dfX = sm.add_constant(weather_train_17_test)
    dfy = pd.DataFrame(np.asarray(result_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 18
- store_nbr == 36

In [ ]:
set_tmax(weather_train_18)
set_tmin(weather_train_18)
set_tavg(weather_train_18)
set_heat_cool(weather_train_18)
merge_heat_cool(weather_train_18)
set_depart(weather_train_18)
set_snowfall_preciptotal(weather_train_18)
set_dewpoint(weather_train_18)
set_wetbulb(weather_train_18)
set_stnpressure(weather_train_18)
set_sealevel(weather_train_18)
set_resultspeed(weather_train_18)
set_resultdir(weather_train_18)
set_avgspeed(weather_train_18)

# merged_18 = merge_weather_train(weather_train_18, train_new_36)
item_nbr_list_18 = get_item_nbr(train_new_36)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_18 :
    plt.subplot(len(item_nbr_list_18), 1, plotcount)
    plt.scatter(x = merged_18['tmax'], y = merged_18[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_18_test = weather_train_18.drop(['station_nbr', 'date', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

train_new_36_pivot = train_new_36.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# train_new_36_pivot.loc[len(train_new_36)] = 0
# train_new_36_pivot.reset_index(drop = True, inplace = True)

# test = train_new_36.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
train_new_36_pivot = pd.DataFrame(np.insert(train_new_36_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_18 :
    dfX = sm.add_constant(weather_train_18_test)
    dfy = pd.DataFrame(np.asarray(train_new_36_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 19
- store_nbr == 30

In [ ]:
set_tmax(weather_train_19)
set_tmin(weather_train_19)
set_tavg(weather_train_19)
set_heat_cool(weather_train_19)
merge_heat_cool(weather_train_19)
set_depart(weather_train_19)
set_snowfall_preciptotal(weather_train_19)
set_dewpoint(weather_train_19)
set_wetbulb(weather_train_19)
set_stnpressure(weather_train_19)
set_sealevel(weather_train_19)
set_resultspeed(weather_train_19)
set_resultdir(weather_train_19)
set_avgspeed(weather_train_19)

# merged_19 = merge_weather_train(weather_train_19, train_new_30)
item_nbr_list_19 = get_item_nbr(train_new_30)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_19 :
    plt.subplot(len(item_nbr_list_19), 1, plotcount)
    plt.scatter(x = merged_19['tmax'], y = merged_19[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_19_test = weather_train_19.drop(['station_nbr', 'date', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

train_new_30_pivot = train_new_30.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# train_new_30_pivot.loc[len(train_new_30)] = 0
# train_new_30_pivot.reset_index(drop = True, inplace = True)

# test = train_new_30.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
train_new_30_pivot = pd.DataFrame(np.insert(train_new_30_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_19 :
    dfX = sm.add_constant(weather_train_19_test)
    dfy = pd.DataFrame(np.asarray(train_new_30_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### station_nbr == 20
- store_nbr == 17
- no depart value

In [ ]:
set_tmax(weather_train_20)
set_tmin(weather_train_20)
set_tavg(weather_train_20)
set_heat_cool(weather_train_20)
merge_heat_cool(weather_train_20)
set_depart(weather_train_20)
set_snowfall_preciptotal(weather_train_20)
set_dewpoint(weather_train_20)
set_wetbulb(weather_train_20)
set_stnpressure(weather_train_20)
set_sealevel(weather_train_20)
set_resultspeed(weather_train_20)
set_resultdir(weather_train_20)
set_avgspeed(weather_train_20)

# merged_20 = merge_weather_train(weather_train_20, train_new_17)
item_nbr_list_20 = get_item_nbr(train_new_17)

In [ ]:
plt.figure(figsize = (10, 20))
plotcount = 1
for num in item_nbr_list_20 :
    plt.subplot(len(item_nbr_list_20), 1, plotcount)
    plt.scatter(x = merged_20['tmax'], y = merged_20[num]) # x에 들어가는 column을 바꿔가며 관계시각화 가능..pairplot은 왜 안되지 ㅅㅂ
    
    plt.ylabel('log_units')
#     plt.ylabel('log_units of item_nbr == {}'.format(num))

    plt.title('item_nbr == {}'.format(num))
        
    plotcount +=1

In [ ]:
weather_train_20_test = weather_train_20.drop(['station_nbr', 'date', 'depart', 'sunrise', 'sunset', 'codesum', 'heat', 'cool'], axis = 1)

train_new_17_pivot = train_new_17.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
# train_new_17_pivot.loc[len(train_new_17)] = 0
# train_new_17_pivot.reset_index(drop = True, inplace = True)

# test = train_new_17.pivot_table(values = 'log_units', index = ['date'], columns = ['item_nbr'])
train_new_17_pivot = pd.DataFrame(np.insert(train_new_17_pivot.values, 359, values = 0, axis = 0))

for num in item_nbr_list_20 :
    dfX = sm.add_constant(weather_train_20_test)
    dfy = pd.DataFrame(np.asarray(train_new_17_pivot[num - 1]), columns=["item_nbr == {}".format(num)])

    model = sm.OLS(dfy.astype(float), dfX.astype(float), missing = 'drop')
    result_model = model.fit()
    print(result_model.summary())

### LinearRegression, OLS

In [ ]:
tmp = merged_1.drop(['station_nbr', 'date', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'snowfall',
                    'preciptotal', 'stnpressure', 'sealevel', 'codesum', 'sunrise', 'sunset', 'resultspeed',
                    'resultdir', 'avgspeed', 'tavg', 'heat/cool'], axis = 1)
tmp_data = tmp.drop([9, 28, 40, 51, 89, 93, 99], axis = 1) # tmax, tmin, tavg, heat/cool column만 있음..일단 이렇게만..
tmp_target = tmp[9] # item_nbr == 9 인 case에 대해서만 우선 해보자
tmp_data_array = np.asarray(weather_train_1)
tmp_target_array = np.asarray(tmp_target)
tmp_data.columns

In [ ]:
# linear regression에서는 array형태 data type이 필요하다.. np.asarray

model_tmp = LinearRegression().fit(tmp_data_array, tmp_target_array)

# model_tmp.coef_
# model_tmp.intercept_

predictions = model_tmp.predict(tmp_data_array)

plt.scatter(tmp_target_array, predictions)
# plt.xlabel(u"실제 집값")
# plt.ylabel(u"집값 예측치")
plt.show()

- Dep. Variable : Which variable is the response in the model

- Model : What model you are using in the fit

- Method : How the parameters of the model were calculated

- No. Observations : The number of observations (examples)
- DF Residuals : Degrees of freedom of the residuals. Number of observations – number of parameters
- DF Model : Number of parameters in the model (not including the constant term if present)

#### The right part of the first table shows the goodness of fit

#### The second table reports for each of the coefficients

- R-squared : The coefficient of determination. A statistical measure of how well the regression line approximates the real data points
    
- Adj. R-squared : The above value adjusted based on the number of observations and the degrees-of-freedom of the residuals
- F-statistic : A measure how significant the fit is. The mean squared error of the model divided by the mean squared error of the residuals
- Prob (F-statistic) : The probability that you would get the above statistic, given the null hypothesis that they are unrelated
- Log-likelihood : The log of the likelihood function.
- AIC : The Akaike Information Criterion. Adjusts the log-likelihood based on the number of observations and the complexity of the model.
- BIC : The Bayesian Information Criterion. Similar to the AIC, but has a higher penalty for models with more parameters.


In [ ]:
dfy_new = result_tmp2.predict(dfX)

plt.scatter(dfy, dfy_new)
# plt.xlabel(u"실제 집값")
# plt.ylabel(u"집값 예측치")
plt.show()